### 1. Extract the pics from paper


```
<!-- 1.Extract the raw pics from the paper and save them to the `plot/raw_data` directory. -->

pdfimages -all xxx.pdf subfig

<!-- 2. Merge the pics piece -->

python merge_3pic.py --top subfig-000.png --mid subfig-001.png --bot subfig-002.png -o merged_01.png

<!-- 3. Change the name of the pics -->
```


### 2.Visualize the RBG uncertainty & error

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio
import os
import shutil

def error_map(pred_path, gt_path, out_path,
              cmap='magma', gamma=0.8, dpi=400):
    pred = iio.imread(pred_path).astype(np.float32) / 255.0
    gt   = iio.imread(gt_path).astype(np.float32) / 255.0

    h = min(pred.shape[0], gt.shape[0])
    w = min(pred.shape[1], gt.shape[1])
    pred = pred[:h, :w, :3]
    gt   = gt[:h, :w, :3]

    # err = np.abs(pred - gt).mean(axis=2)   # L1
    err = np.square(pred - gt).mean(axis=2)   # L2
    vmax = np.percentile(err, 99)
    err = np.clip(err / vmax, 0, 1)
    if gamma != 1.0:
        err = np.power(err, gamma) # 504*378

    # reshape to 533x400
    # err = ndimage.zoom(err, (533/504, 400/378), order=1)

    plt.imsave(out_path, err, cmap=cmap)
    print(f"Saved: {out_path}")

def unc_map(un_path, out_path,
              cmap='magma', gamma=0.8, dpi=400):
    unc = iio.imread(un_path).astype(np.float32) / 255.0

    h = unc.shape[0]
    w = unc.shape[1]
    unc = unc[:h, :w, :3]/10.0
    unc = np.square(unc).mean(axis=2)
    vmax = np.percentile(unc, 99)
    unc = np.clip(unc / vmax, 0, 1)
    if gamma != 1.0:
        unc = np.power(unc, gamma)

    # reshape to 533x400
    # unc = ndimage.zoom(unc, (533/504, 400/378), order=1)

    plt.imsave(out_path, unc, cmap=cmap)
    print(f"Saved: {out_path}")

def cpfile(src_path, dst_path):
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)
    print(f"Copied: {src_path} to {dst_path}")

In [14]:
result_path = "/home/fwu/Documents/MyProjects/hs-gs/output/best/llff"

scene_name = "trex" # "fern", "flower", "fortress", "horns", "leaves", "orchids", "room", "trex"

gt_path = "test/ours_7000/gt"
render_path = "test/ours_7000/renders"
unc_path = "test/ours_7000/unc"

no = 1

gt = f"{result_path}/{scene_name}/{gt_path}/{no:05d}.png"
render = f"{result_path}/{scene_name}/{render_path}/{no:05d}.png"
unc = f"{result_path}/{scene_name}/{unc_path}/{no:05d}.png"

cpfile(render, f"llff/{scene_name}_render_{no:05d}.png")
cpfile(gt, f"llff/{scene_name}_gt_{no:05d}.png")

error_map(gt, render, f"llff/{scene_name}_error_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)
unc_map(unc, f"llff/{scene_name}_unc_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)

Copied: /home/fwu/Documents/MyProjects/hs-gs/output/best/llff/trex/test/ours_7000/renders/00001.png to llff/trex_render_00001.png
Copied: /home/fwu/Documents/MyProjects/hs-gs/output/best/llff/trex/test/ours_7000/gt/00001.png to llff/trex_gt_00001.png
Saved: llff/trex_error_00001.png
Saved: llff/trex_unc_00001.png


In [ ]:
# 用法示例

pred = 'best_fern/renders/00000.png'
gt = 'best_fern/gt/00000.png'
error_map(pred, gt, "best_fern/error_vis.png",
               cmap='magma', gamma=1.0, dpi=400)

unc = 'best_fern/unc/00000.png'
unc_map(unc, "best_fern/unc_vis2.png",
               cmap='magma', gamma=1.0, dpi=400)

#### Resize the pics to 504*378

In [1]:
# resize the pics to 504*378

import cv2
import os
from scipy import ndimage
from PIL import Image

# 方法1: 使用OpenCV的INTER_CUBIC (最常用)
def resize_bicubic_cv2(image, target_size):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_CUBIC)

target_size = (504, 378)
save_dir = 'render_result'

for file in os.listdir('extract_pics'):
    image = cv2.imread(f'extract_pics/{file}')
    resized_image = resize_bicubic_cv2(image, target_size)
    cv2.imwrite(f'{save_dir}/{file}', resized_image)

print('Done.')

Done.


#### Visulization the depth

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


def error_map(depth_gt, depth_render, out_path, cmap='magma', gamma=1.0, dpi=400):
    err = np.abs(depth_render - depth_gt)
    vmax = np.percentile(err, 99)
    if vmax > 0:  # 避免除零
        err = np.clip(err / vmax, 0, 1)
    if gamma != 1.0:
        err = np.power(err, gamma)
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.imsave(out_path, err, cmap=cmap, dpi=dpi)
    print(f"Saved: {out_path}")

def unc_map(depth_std, out_path, cmap='magma', gamma=1.0, dpi=400):
    unc = np.abs(depth_std)
    vmax = np.percentile(unc, 99)
    if vmax > 0:  # 避免除零
        unc = np.clip(unc / vmax, 0, 1)
    if gamma != 1.0:
        unc = np.power(unc, gamma)
    
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.imsave(out_path, unc, cmap=cmap, dpi=dpi)
    print(f"Saved: {out_path}")

def depth_map(depth, out_path, cmap='magma', gamma=1.0, dpi=400):
    depth = np.abs(depth)
    vmax = np.percentile(depth, 99)
    if vmax > 0:  # 避免除零
        depth = np.clip(depth / vmax, 0, 1)
    if gamma != 1.0:
        depth = np.power(depth, gamma)
    
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.imsave(out_path, depth, cmap=cmap, dpi=dpi)
    print(f"Saved: {out_path}")

depth_index = {
    "statue": [68, 70, 72, 74],
    "africa": [0, 8, 10, 12],
    "torch": [9, 11, 13, 15],
    "basket": [42, 44, 46, 48]
}

# unc_index = ['00000', '00001', '00002', '00003']

root_path = "/disk1/fwu/3DGS/LF"
result_path = "/home/fwu/Documents/MyProjects/hs-gs/output/best/LF"

scene_name = "torch" # "africa", "basket", "statue", "torch"
depth_unc_path = "test/ours_3000/depth_uncertainty"

no = 0

depth_gt = np.load(f'{root_path}/{scene_name}/depth_gt_{depth_index[scene_name][no]:02d}.npy')  # shape: (H, W)
depth_unc = np.load(f'{result_path}/{scene_name}/{depth_unc_path}/{no:05d}.npy')  # shape: (H, W)

depth_render = depth_unc.mean(axis=0).squeeze(0)
depth_std = depth_unc.std(axis=0).squeeze(0)

depth_render = cv2.resize(depth_render, (depth_gt.shape[1], depth_gt.shape[0]), interpolation=cv2.INTER_NEAREST)
depth_std = cv2.resize(depth_std, (depth_gt.shape[1], depth_gt.shape[0]), interpolation=cv2.INTER_NEAREST)


# 处理无效值
depth_gt = np.nan_to_num(depth_gt, nan=0.0, posinf=0.0, neginf=0.0)
depth_render = np.nan_to_num(depth_render, nan=0.0, posinf=0.0, neginf=0.0)
depth_std = np.nan_to_num(depth_std, nan=0.0, posinf=0.0, neginf=0.0)

error_map(depth_gt, depth_render, f"LF/{scene_name}_error_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)
unc_map(depth_std, f"LF/{scene_name}_unc_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)
depth_map(depth_gt, f"LF/{scene_name}_gt_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)
depth_map(depth_render, f"LF/{scene_name}_render_{no:05d}.png", cmap='magma', gamma=1.0, dpi=400)

Saved: LF/torch_error_00000.png
Saved: LF/torch_unc_00000.png
Saved: LF/torch_gt_00000.png
Saved: LF/torch_render_00000.png
